# Bhagavad Gita AI Assistant 🕉️

This notebook implements a RAG (Retrieval-Augmented Generation) system that:
1. **Loads** all 700+ verses from the Bhagavad Gita API
2. **Stores** them in a vector database (ChromaDB) with semantic embeddings
3. **Retrieves** relevant slokas based on user problems
4. **Generates** wisdom-based solutions using an LLM

## Architecture
```
User Problem → Semantic Search → Top-K Slokas → LLM Reasoning → Solution
```

## 1. Setup & Configuration

Install dependencies first:
```bash
pip install -r requirements.txt
```

In [ ]:
import requests
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import time
import json
from typing import List, Dict, Optional
import os
from pathlib import Path
import ollama

print("✅ All imports successful!")

In [ ]:
# Configuration
DB_PATH = "./bg_vector_db"
COLLECTION_NAME = "bhagavad_gita"
EMBEDDING_MODEL = 'paraphrase-multilingual-MiniLM-L12-v2'  # Supports English and Sanskrit
MAX_RETRIES = 3
RETRY_DELAY = 2  # seconds

# Chapter structure (number of verses per chapter)
# Source: https://github.com/vedicscriptures/bhagavad-gita
CHAPTER_VERSES = {
    1: 47, 2: 72, 3: 43, 4: 42, 5: 29, 6: 47,
    7: 30, 8: 28, 9: 34, 10: 42, 11: 55, 12: 20,
    13: 35, 14: 27, 15: 20, 16: 24, 17: 28, 18: 78
}

# Ollama Configuration (running in Kubernetes)
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://ollama.ollama.svc.cluster.local:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2")
print(f"📁 Database path: {DB_PATH}")
print(f"🤖 Embedding model: {EMBEDDING_MODEL}")
print(f"🦙 Ollama URL: {OLLAMA_BASE_URL}")
print(f"🧠 Ollama model: {OLLAMA_MODEL}")
print(f"📚 Total verses to load: {sum(CHAPTER_VERSES.values())}")

## 2. Data Loading Functions

Enhanced version with:
- Error handling and retries
- Progress tracking
- Rich metadata extraction

In [ ]:
def fetch_verse(chapter: int, verse: int, max_retries: int = MAX_RETRIES) -> Optional[Dict]:
    """
    Fetch a single verse from the Bhagavad Gita API with retry logic.
    
    Args:
        chapter: Chapter number (1-18)
        verse: Verse number
        max_retries: Maximum number of retry attempts
    
    Returns:
        Dictionary containing verse data or None if failed
    """
    url = f"https://vedicscriptures.github.io/slok/{chapter}/{verse}/"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 404:
                # Verse doesn't exist, don't retry
                return None
            else:
                # Other error, retry
                if attempt < max_retries - 1:
                    time.sleep(RETRY_DELAY)
                continue
                
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                time.sleep(RETRY_DELAY)
            else:
                print(f"\n❌ Failed to fetch {chapter}.{verse} after {max_retries} attempts: {e}")
                return None
    
    return None


def extract_verse_data(data: Dict, chapter: int, verse: int) -> Dict:
    """
    Extract and structure relevant data from API response.
    
    Returns:
        Dictionary with structured verse data
    """
    # Build comprehensive search text combining all translations and commentaries
    search_components = []
    
    # Add English translation (Sivananda)
    siva_translation = data.get('siva', {}).get('et', '')
    if siva_translation:
        search_components.append(siva_translation)
    
    # Add Sivananda commentary
    siva_commentary = data.get('siva', {}).get('ec', '')
    if siva_commentary:
        search_components.append(siva_commentary)
    
    # Add Chinmayananda commentary (often very insightful)
    chinmay_commentary = data.get('chinmay', {}).get('ec', '')
    if chinmay_commentary:
        search_components.append(chinmay_commentary)
    
    # Add Purohit Swami translation (simpler English)
    purohit_translation = data.get('purohit', {}).get('et', '')
    if purohit_translation:
        search_components.append(purohit_translation)
    
    # Combine all components
    search_text = " ".join(search_components)
    
    # Extract metadata
    metadata = {
        "chapter": chapter,
        "verse": verse,
        "verse_id": f"{chapter}.{verse}",
        "sanskrit": data.get("slok", ""),
        "transliteration": data.get("transliteration", ""),
        "translation": siva_translation,
        "commentary": siva_commentary[:500] if siva_commentary else "",  # Truncate for metadata
    }
    
    return {
        "id": f"{chapter}.{verse}",
        "search_text": search_text,
        "metadata": metadata
    }


print("✅ Data loading functions defined")

## 3. Initialize Vector Database

Setup ChromaDB with persistent storage and embedding model.

In [ ]:
# Initialize ChromaDB client
print("🔄 Initializing ChromaDB...")
client = chromadb.PersistentClient(path=DB_PATH)

# Delete existing collection if you want to reload data
# Uncomment the following lines to reset the database
# try:
#     client.delete_collection(name=COLLECTION_NAME)
#     print("🗑️ Deleted existing collection")
# except:
#     pass

# Get or create collection
collection = client.get_or_create_collection(name=COLLECTION_NAME)

print(f"✅ Collection '{COLLECTION_NAME}' ready")
print(f"📊 Current collection size: {collection.count()} verses")

In [ ]:
# Initialize embedding model
print("🔄 Loading embedding model...")
model = SentenceTransformer(EMBEDDING_MODEL)
print(f"✅ Embedding model loaded: {EMBEDDING_MODEL}")

## 4. Load All Verses into Vector Database

This will fetch all verses from the API and store them with semantic embeddings.

**Note:** This takes about 5-10 minutes to complete. Run it once, and the data will be persisted.

In [ ]:
# Check if we need to load data
current_count = collection.count()
expected_count = sum(CHAPTER_VERSES.values())

if current_count >= expected_count:
    print(f"✅ Database already contains {current_count} verses. Skipping data load.")
    print("   To reload data, delete the collection in the previous cell.")
else:
    print(f"🔄 Loading data into vector database...")
    print(f"   Current: {current_count} verses, Expected: {expected_count} verses\n")
    
    # Track statistics
    total_verses = 0
    failed_verses = []
    
    # Loop through all chapters
    for chapter_num in range(1, 19):
        max_verses = CHAPTER_VERSES[chapter_num]
        chapter_verses_loaded = 0
        
        # Progress bar for this chapter
        pbar = tqdm(range(1, max_verses + 1), 
                   desc=f"Chapter {chapter_num:2d}",
                   unit="verse")
        
        for verse_num in pbar:
            # Check if verse already exists
            verse_id = f"{chapter_num}.{verse_num}"
            try:
                existing = collection.get(ids=[verse_id])
                if existing['ids']:
                    chapter_verses_loaded += 1
                    continue
            except:
                pass
            
            # Fetch verse from API
            data = fetch_verse(chapter_num, verse_num)
            
            if data is None:
                failed_verses.append(f"{chapter_num}.{verse_num}")
                continue
            
            # Extract and structure data
            verse_data = extract_verse_data(data, chapter_num, verse_num)
            
            # Add to vector database
            collection.add(
                ids=[verse_data["id"]],
                documents=[verse_data["search_text"]],
                metadatas=[verse_data["metadata"]]
            )
            
            chapter_verses_loaded += 1
            total_verses += 1
            
            # Small delay to be respectful to the API
            time.sleep(0.1)
        
        pbar.close()
        print(f"   ✅ Chapter {chapter_num}: {chapter_verses_loaded}/{max_verses} verses loaded")
    
    print(f"\n🎉 Data loading complete!")
    print(f"   Total verses in database: {collection.count()}")
    print(f"   New verses loaded: {total_verses}")
    
    if failed_verses:
        print(f"   ⚠️ Failed to load {len(failed_verses)} verses: {failed_verses[:10]}...")

## 5. Semantic Search Function

Query the vector database to find relevant slokas based on semantic similarity.

In [ ]:
def search_slokas(query: str, n_results: int = 5, chapter_filter: Optional[int] = None) -> Dict:
    """
    Search for relevant Bhagavad Gita verses based on semantic similarity.
    
    Args:
        query: User's problem or question
        n_results: Number of top results to return
        chapter_filter: Optional chapter number to filter results (1-18)
    
    Returns:
        Dictionary containing search results with metadata
    """
    # Build where clause for filtering
    where_clause = None
    if chapter_filter:
        where_clause = {"chapter": chapter_filter}
    
    # Perform semantic search
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        where=where_clause
    )
    
    return results


def format_search_results(results: Dict, show_commentary: bool = True) -> str:
    """
    Format search results for display.
    
    Args:
        results: Results from search_slokas()
        show_commentary: Whether to include commentary in output
    
    Returns:
        Formatted string with verse information
    """
    output = []
    
    for i, metadata in enumerate(results['metadatas'][0]):
        verse_id = metadata['verse_id']
        chapter = metadata['chapter']
        verse = metadata['verse']
        
        output.append(f"\n{'='*80}")
        output.append(f"📖 Bhagavad Gita {verse_id} (Chapter {chapter}, Verse {verse})")
        output.append(f"{'='*80}")
        
        # Sanskrit
        output.append(f"\n🕉️ **Sanskrit:**\n{metadata['sanskrit']}")
        
        # Transliteration
        if metadata.get('transliteration'):
            output.append(f"\n📝 **Transliteration:**\n{metadata['transliteration']}")
        
        # Translation
        output.append(f"\n🌍 **Translation:**\n{metadata['translation']}")
        
        # Commentary (optional)
        if show_commentary and metadata.get('commentary'):
            output.append(f"\n💡 **Commentary:**\n{metadata['commentary'][:300]}...")
    
    return "\n".join(output)


print("✅ Search functions defined")

## 6. Test Semantic Search

Let's test the search functionality with a sample query.

In [ ]:
# Test search
test_query = "How to overcome fear and anxiety?"

print(f"🔍 Searching for: '{test_query}'\n")
results = search_slokas(test_query, n_results=3)

print(format_search_results(results, show_commentary=False))

## 7. LLM Integration

Generate wisdom-based solutions using retrieved slokas and an LLM.

In [ ]:
def get_llm_response_ollama(prompt: str, base_url: str = OLLAMA_BASE_URL, model: str = OLLAMA_MODEL) -> str:
    """
    Get response from local Ollama LLM running in Kubernetes.
    
    Args:
        prompt: The formatted prompt
        base_url: Ollama base URL
        model: Model name to use
    
    Returns:
        LLM response string
    """
    try:
        # Configure Ollama client to use Kubernetes service
        client = ollama.Client(host=base_url)
        
        response = client.chat(
            model=model,
            messages=[
                {
                    "role": "system", 
                    "content": "You are a compassionate spiritual guide well-versed in the Bhagavad Gita."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            options={
                "temperature": 0.7,
                "num_predict": 800
            }
        )
        
        return response['message']['content']
    
    except Exception as e:
        return f"⚠️ Error calling Ollama API at {base_url}: {str(e)}\n\nPlease ensure:\n1. Ollama is running in your Kubernetes cluster\n2. The service is accessible at {base_url}\n3. Model '{model}' is available"


print("✅ Ollama LLM integration ready!")

In [ ]:
def create_prompt(problem: str, slokas_data: Dict) -> str:
    """
    Create a prompt for the LLM combining the user's problem and retrieved slokas.
    
    Args:
        problem: User's problem or question
        slokas_data: Results from semantic search
    
    Returns:
        Formatted prompt string
    """
    # Extract slokas information
    slokas = []
    for metadata in slokas_data['metadatas'][0]:
        sloka_info = f"""
Verse {metadata['verse_id']}:
Sanskrit: {metadata['sanskrit']}
Translation: {metadata['translation']}
Commentary: {metadata.get('commentary', 'N/A')}
"""
        slokas.append(sloka_info)
    
    slokas_text = "\n---\n".join(slokas)
    
    prompt = f"""You are a wise spiritual guide knowledgeable in the Bhagavad Gita. A person has come to you with the following problem:

**Problem:** {problem}

Based on your deep understanding of the Bhagavad Gita, you have identified the following relevant verses:

{slokas_text}

**Instructions:**
1. Provide a compassionate and insightful response to the person's problem
2. Reference specific verses (by chapter.verse number) that are relevant
3. Explain the wisdom from these verses in a way that directly addresses their concern
4. Offer practical guidance based on the teachings
5. Keep your response concise (250-400 words) but meaningful
6. Use a warm, supportive tone

Please provide your guidance:"""
    
    return prompt


print("✅ Prompt engineering function defined")

## 8. Complete RAG Pipeline

Combine everything into a single function that takes a problem and returns a solution.

In [ ]:
def solve_problem_with_gita(problem: str, 
                            n_slokas: int = 5, 
                            show_slokas: bool = True) -> str:
    """
    Complete RAG pipeline: Search for slokas + Generate solution using local Ollama.
    
    Args:
        problem: User's problem or question
        n_slokas: Number of slokas to retrieve
        show_slokas: Whether to display retrieved slokas
    
    Returns:
        Formatted output with slokas and LLM-generated solution
    """
    print("🔍 Step 1: Searching for relevant slokas...\n")
    
    # Search for relevant slokas
    slokas_data = search_slokas(problem, n_results=n_slokas)
    
    if show_slokas:
        print(format_search_results(slokas_data, show_commentary=False))
        print("\n" + "="*80 + "\n")
    
    # Create prompt
    print("🤖 Step 2: Generating wisdom-based solution with Ollama...\n")
    prompt = create_prompt(problem, slokas_data)
    
    # Get LLM response from local Ollama
    solution = get_llm_response_ollama(prompt)
    
    # Format final output
    output = f"""
{'='*80}
💡 GUIDANCE FROM THE BHAGAVAD GITA
{'='*80}

{solution}

{'='*80}
🕉️ May the wisdom of the Gita guide your path
{'='*80}
"""
    
    return output


print("✅ Complete RAG pipeline ready!")

## 9. Interactive Demo

Try the complete system with real problems!

### Example 1: Career Confusion

In [ ]:
problem1 = """
I'm confused about my career path. I have a stable job, but I'm not passionate about it. 
My parents want me to continue in this field, but I dream of doing something different. 
I'm afraid of making the wrong choice and disappointing everyone. What should I do?
"""

response = solve_problem_with_gita(problem1, n_slokas=4,  show_slokas=True)
print(response)

### Example 2: Managing Anger

In [ ]:
problem2 = """
I get angry very easily, especially when things don't go my way at work. 
This anger is affecting my relationships and my peace of mind. 
How can I control my anger and remain calm?
"""

response = solve_problem_with_gita(problem2, n_slokas=4,  show_slokas=True)
print(response)

### Example 3: Dealing with Loss

In [ ]:
problem3 = """
I recently lost a loved one and I'm struggling to cope with the grief. 
I feel lost and don't know how to move forward. 
How can I find peace and accept this loss?
"""

response = solve_problem_with_gita(problem3, n_slokas=5,  show_slokas=False)
print(response)

### Try Your Own Problem!

In [ ]:
# Enter your own problem here
my_problem = """
I feel overwhelmed by all my responsibilities - work, family, health. 
I don't have time for myself and feel burnt out. How can I find balance?
"""

response = solve_problem_with_gita(my_problem, n_slokas=4,  show_slokas=True)
print(response)

## 10. Advanced Queries

Some helper functions for exploring the database.

In [ ]:
def get_verse_by_id(chapter: int, verse: int) -> str:
    """
    Get a specific verse by chapter and verse number.
    """
    verse_id = f"{chapter}.{verse}"
    result = collection.get(ids=[verse_id])
    
    if result['ids']:
        metadata = result['metadatas'][0]
        output = f"""
📖 Bhagavad Gita {verse_id}
{'='*80}

🕉️ Sanskrit:
{metadata['sanskrit']}

🌍 Translation:
{metadata['translation']}

💡 Commentary:
{metadata.get('commentary', 'N/A')}
"""
        return output
    else:
        return f"❌ Verse {verse_id} not found in database."


# Example: Get the famous verse 2.47
print(get_verse_by_id(2, 47))

In [ ]:
def search_by_chapter(query: str, chapter: int, n_results: int = 3) -> str:
    """
    Search within a specific chapter only.
    """
    print(f"🔍 Searching in Chapter {chapter} for: '{query}'\n")
    results = search_slokas(query, n_results=n_results, chapter_filter=chapter)
    return format_search_results(results, show_commentary=False)


# Example: Search for verses about dharma in Chapter 2
print(search_by_chapter("duty and dharma", chapter=2, n_results=3))

## 11. Database Statistics

In [ ]:
# Get statistics about the database
total_verses = collection.count()

print("📊 Bhagavad Gita Vector Database Statistics")
print("=" * 80)
print(f"Total verses in database: {total_verses}")
print(f"Expected total verses: {sum(CHAPTER_VERSES.values())}")
print(f"\nChapter-wise verse count:")

for chapter in range(1, 19):
    count = len(collection.get(where={"chapter": chapter})['ids'])
    expected = CHAPTER_VERSES[chapter]
    status = "✅" if count == expected else "⚠️"
    print(f"  Chapter {chapter:2d}: {count:2d}/{expected:2d} verses {status}")

print("\n" + "="*80)

## 12. Next Steps

### Enhancements You Can Make:

1. **Web Interface**: Build a Streamlit or Gradio interface for easier interaction
2. **Multiple Languages**: Add support for Hindi, Sanskrit responses
3. **Context Memory**: Implement conversation history for follow-up questions
4. **Fine-tuning**: Fine-tune the embedding model on Gita-specific corpus
5. **Audio Output**: Add text-to-speech for verse recitation
6. **Daily Verse**: Create a feature to get a random verse daily
7. **Verse Comparison**: Compare interpretations from different commentators
8. **Mind Map**: Visualize connections between verses on similar topics

### Configuration Tips:

1. **API Keys**: Set environment variables for security:
   ```bash
   export GROQ_API_KEY="your_key_here"
   export OPENAI_API_KEY="your_key_here"
   ```

2. **Local LLM**: Use Ollama for completely local operation:
   ```bash
   ollama pull llama3.2
   ```

3. **Tuning Search**: Adjust `n_slokas` parameter (3-7 works best)

4. **Database Reset**: To reload data from scratch, delete the `bg_vector_db` folder

### Resources:

- [Bhagavad Gita API Docs](https://github.com/vedicscriptures/bhagavad-gita)
- [ChromaDB Documentation](https://docs.trychroma.com/)
- [Sentence Transformers](https://www.sbert.net/)
- [Groq API](https://console.groq.com/)

🕉️ **May this tool help bring the timeless wisdom of the Bhagavad Gita to those who seek it!**